In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import pickle

In [3]:
df_movies = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\movies_dataset.csv', encoding='utf-8')
df_credits = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\credits.csv', encoding='utf-8')

C:\Users\house\AppData\Local\Temp\ipykernel_26148\204366351.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\movies_dataset.csv', encoding='utf-8')


In [4]:
print(df_movies.shape)
df_movies.head(1)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [5]:
print(df_credits.shape)
df_credits.head(1)

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [6]:
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')
df_movies.dropna(subset=['id'], inplace=True)
df_movies['id'].astype('int64')

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45463, dtype: int64

In [7]:
df_joined = df_movies.merge(df_credits, on='id', how='outer')
print(df_joined.shape)
df_joined.head(1)

(45539, 26)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,2.0,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,69.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}, {'iso_6...",Released,NaN,Ariel,False,7.1,44.0,"[{'cast_id': 3, 'character': 'Taisto Olavi Kas...","[{'credit_id': '52fe420dc3a36847f800001f', 'de..."


In [8]:
print(df_joined.shape)
df_joined.columns

(45539, 26)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [9]:
df_eda = df_joined[['id', 'belongs_to_collection', 'genres', 'overview', 'title', 'cast', 'crew']]
df_eda.dropna(inplace=True)
print(df_eda.shape)
df_eda.head(1)

(4440, 7)


C:\Users\house\AppData\Local\Temp\ipykernel_26148\1100454394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda.dropna(inplace=True)


,id,belongs_to_collection,genres,overview,title,cast,crew
4,11.0,"{'id': 10, 'name': 'Star Wars Collection', 'po...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Princess Leia is captured and held hostage by ...,Star Wars,"[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de..."


In [10]:
def extraer(texto):
    lista = []    
    datos = ast.literal_eval(texto)    
    if isinstance(datos, list):        
        for item in datos:
            if isinstance(item, dict) and 'name' in item:
                lista.append(item['name'])    
    elif isinstance(datos, dict) and 'name' in datos:
        lista.append(datos['name'])    
    return lista


def directores(texto):
    lista = []    
    for i in ast.literal_eval(texto):
        if i['job'] == 'Director':
            lista.append(i['name'])
    return lista

def del_espacios(texto):
    lista = []
    for i in texto:
        lista.append(i.replace(" ", ""))
    return lista


In [11]:
df_eda['genres'] = df_eda['genres'].apply(extraer)
df_eda['cast'] = df_eda['cast'].apply(extraer)
df_eda['cast'] = df_eda['cast'].apply(lambda x: x[0:3])
df_eda['belongs_to_collection'] = df_eda['belongs_to_collection'].apply(extraer)
df_eda['crew'] = df_eda['crew'].apply(directores)
df_eda['overview'] = df_eda['overview'].apply(lambda x: x.split())

df_eda['belongs_to_collection'] = df_eda['belongs_to_collection'].apply(del_espacios)
df_eda['genres'] = df_eda['genres'].apply(del_espacios)
df_eda['cast'] = df_eda['cast'].apply(del_espacios)
df_eda['crew'] = df_eda['crew'].apply(del_espacios)

df_eda['overview'] = df_eda['overview'].apply(lambda x: list(filter(lambda y: y and y.strip(), x)))

C:\Users\house\AppData\Local\Temp\ipykernel_26148\2248557828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda['genres'] = df_eda['genres'].apply(extraer)
C:\Users\house\AppData\Local\Temp\ipykernel_26148\2248557828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda['cast'] = df_eda['cast'].apply(extraer)
C:\Users\house\AppData\Local\Temp\ipykernel_26148\2248557828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [12]:
df_eda['tags'] = df_eda['belongs_to_collection'] + df_eda['genres'] + df_eda['overview'] + df_eda['cast'] + df_eda['crew']

df_eda.head()

C:\Users\house\AppData\Local\Temp\ipykernel_26148\727673074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda['tags'] = df_eda['belongs_to_collection'] + df_eda['genres'] + df_eda['overview'] + df_eda['cast'] + df_eda['crew']


,id,belongs_to_collection,genres,overview,title,cast,crew,tags
4,11.0,[StarWarsCollection],"[Adventure, Action, ScienceFiction]","[Princess, Leia, is, captured, and, held, host...",Star Wars,"[MarkHamill, HarrisonFord, CarrieFisher]",[GeorgeLucas],"[StarWarsCollection, Adventure, Action, Scienc..."
5,12.0,[FindingNemoCollection],"[Animation, Family]","[Nemo,, an, adventurous, young, clownfish,, is...",Finding Nemo,"[AlbertBrooks, EllenDeGeneres, AlexanderGould]",[AndrewStanton],"[FindingNemoCollection, Animation, Family, Nem..."
9,16.0,[HeartofGoldCollection],"[Drama, Crime, Music]","[Selma,, a, Czech, immigrant, on, the, verge, ...",Dancer in the Dark,"[Björk, CatherineDeneuve, DavidMorse]",[LarsvonTrier],"[HeartofGoldCollection, Drama, Crime, Music, S..."
14,21.0,[TheEndlessSummerCollection],[Documentary],"[The, Endless, Summer,, by, Bruce, Brown,, is,...",The Endless Summer,"[MichaelHynson, RobertAugust, Lord'TallyHo'Ble...",[BruceBrown],"[TheEndlessSummerCollection, Documentary, The,..."
15,22.0,[PiratesoftheCaribbeanCollection],"[Adventure, Fantasy, Action]","[Jack, Sparrow,, a, freewheeling, 17th-century...",Pirates of the Caribbean: The Curse of the Bla...,"[JohnnyDepp, GeoffreyRush, OrlandoBloom]",[GoreVerbinski],"[PiratesoftheCaribbeanCollection, Adventure, F..."


In [ ]:
df_for_analisys = df_eda.drop(columns=['overview', 'belongs_to_collection', 'genres', 'cast', 'crew'])
df_for_analisys.reset_index(drop=True, inplace=True)


In [14]:
df_for_analisys['tags'] = df_for_analisys['tags'].apply(lambda x: ' '.join(x))
df_for_analisys.head()

,id,title,tags
4,11.0,Star Wars,StarWarsCollection Adventure Action ScienceFic...
5,12.0,Finding Nemo,"FindingNemoCollection Animation Family Nemo, a..."
9,16.0,Dancer in the Dark,"HeartofGoldCollection Drama Crime Music Selma,..."
14,21.0,The Endless Summer,TheEndlessSummerCollection Documentary The End...
15,22.0,Pirates of the Caribbean: The Curse of the Bla...,PiratesoftheCaribbeanCollection Adventure Fant...


In [22]:
print(df_for_analisys['tags'].iloc[0])

StarWarsCollection Adventure Action ScienceFiction Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire MarkHamill HarrisonFord CarrieFisher GeorgeLucas


In [21]:
df_for_analisys['tags'] = df_for_analisys['tags'].str.replace(r'[.,]', '', regex=True)

In [44]:
df_for_analisys.to_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\dataset_recomendaciones.csv', index=False)

Creamos el modelo

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(df_for_analisys['tags']).toarray()
vector.shape

(4440, 5000)

In [28]:
similaridad = cosine_similarity(vector)
similaridad

array([[1.        , 0.        , 0.        , ..., 0.        , 0.02993422,
        0.04134491],
       [0.        , 1.        , 0.        , ..., 0.        , 0.07658396,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.04428074, 0.03279129,
        0.04529108],
       ...,
       [0.        , 0.        , 0.04428074, ..., 1.        , 0.04356068,
        0.06016568],
       [0.02993422, 0.07658396, 0.03279129, ..., 0.04356068, 1.        ,
        0.0222773 ],
       [0.04134491, 0.        , 0.04529108, ..., 0.06016568, 0.0222773 ,
        1.        ]])

In [37]:
df_for_analisys[df_for_analisys['title'] == 'Star Wars'].index[0]

np.int64(0)

In [39]:
def recomendacion(pelicula:str):
    indice = df_for_analisys[df_for_analisys['title'] == pelicula].index[0]

    distancia = sorted(list(enumerate(similaridad[indice])), reverse=True, key=lambda x: x[1])

    for i in distancia[1:6]:
        print(df_for_analisys.iloc[i[0]].title)

In [42]:
recomendacion('Batman')

Batman Beyond: Return of the Joker
The Dark Knight Rises
Batman & Robin
Batman Unlimited: Monster Mayhem
LEGO DC Comics Super Heroes: Justice League - Gotham City Breakout


In [41]:
pickle.dump(df_for_analisys, open('peliculas.pkl', 'wb'))
pickle.dump(similaridad, open('similaridad.pkl', 'wb'))